# AWD-LSTM with target language model pretraining

In this notebook we train an AWD-LSTM model for the proxy task with language model pretraining on the target data.  This notebook has been adapted from the fast.ai [ULMFit tutorial](https://github.com/fastai/course-nlp/blob/master/nn-vietnamese.ipynb).

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai import *
from fastai.text import *
import glob
import eval_models
import pandas as pd

In [ ]:
bs=48

In [ ]:
torch.cuda.set_device(1)
torch.__version__

In [ ]:
data_path = Config.data_path()

In [ ]:
name = 'bscore_lm'
path = data_path/name
path.mkdir(exist_ok=True, parents=True)

### Target Language Model

In [ ]:
data = load_data(path, 'lm_target_databunch', bs=bs)

In [ ]:
learn = language_model_learner(data, AWD_LSTM, drop_mult=0.5, pretrained=False)

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
lr = 3e-3

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(1, lr, moms=(0.8,0.7))

In [ ]:
learn.fit_one_cycle(10, lr, moms=(0.8,0.7))

In [ ]:
learn.fit_one_cycle(5, lr, moms=(0.8,0.7))

In [ ]:
learn.fit_one_cycle(5, lr, moms=(0.8,0.7))

In [ ]:
mdl_path = path/'models'
mdl_path.mkdir(exist_ok=True)

In [ ]:
model_basename = 'awdlstm_train-target_lm'
learn.save(mdl_path/f'{model_basename}_wt', with_opt=False)
learn.data.vocab.save(mdl_path/f'{model_basename}_vocab.pkl')
learn.save_encoder(mdl_path/f'{model_basename}enc')

### Train Classifier

In [ ]:
train_df = pd.read_csv(path/'train64.csv')
valid_df = pd.read_csv(path/'valid64.csv')
test_df = pd.read_csv(path/'test64.csv')

In [ ]:
basicTokenizer = Tokenizer(pre_rules=[], post_rules=[])
data_clas = TextDataBunch.from_df(path, train_df, valid_df, tokenizer=basicTokenizer,
                                  vocab = data.vocab, bs=bs, num_workers=1)

In [ ]:
learn_c = text_classifier_learner(data_clas, AWD_LSTM, pretrained=False, drop_mult=0.5, 
                                  metrics=[accuracy, FBeta(average='macro', beta=1)])

In [ ]:
learn_c.load_encoder(f'{model_basename}enc')

In [ ]:
learn_c.freeze()

In [ ]:
learn_c.lr_find()

In [ ]:
learn_c.recorder.plot()

In [ ]:
lr = 1e-4

In [ ]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

In [ ]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

In [ ]:
learn_c.freeze_to(-2)
learn_c.fit_one_cycle(2, slice(lr/(2.6**4),lr), moms=(0.8,0.7))

In [ ]:
learn_c.freeze_to(-3)
learn_c.fit_one_cycle(2, slice(lr/2/(2.6**4),lr/2), moms=(0.8,0.7))

In [ ]:
learn_c.unfreeze()
learn_c.fit_one_cycle(1, slice(lr/10/(2.6**4),lr/10), moms=(0.8,0.7))

In [ ]:
learn_c.save(mdl_path/'awdlstm_train-target_clas')

In [ ]:
learn_c.load(mdl_path/'new_models'/'awdlstm_train-target_clas-size64-augment7')

### Evaluate Classifier

Evaluate on the proxy task -- classifying fixed-length chunks of bootleg score features.

In [ ]:
data_clas_test = TextDataBunch.from_df(path, train_df, test_df, tokenizer=basicTokenizer, 
                                       vocab=data.vocab, bs=bs, num_workers=1)

In [ ]:
learn_c.validate(data_clas_test.valid_dl)

Evaluate on the original task -- classifying pages of sheet music.  We can evaluate our models in two ways:
- applying the model to a variable length sequence
- applying the model to multiple fixed-length windows and averaging the predictions

First we evaluate the model on variable length inputs.  Report results with and without applying priors.

In [ ]:
train_fullpage_df = pd.read_csv(path/'train.fullpage.csv')
valid_fullpage_df = pd.read_csv(path/'valid.fullpage.csv')
test_fullpage_df = pd.read_csv(path/'test.fullpage.csv')

In [ ]:
(acc, acc_with_prior), (f1, f1_with_prior) = eval_models.calcAccuracy_fullpage(learn_c, path, train_fullpage_df, valid_fullpage_df, test_fullpage_df)
(acc, acc_with_prior), (f1, f1_with_prior)

Now we evaluate the model by considering multiple fixed-length windows and averaging the predictions.

In [ ]:
test_ensemble_df = pd.read_csv(path/'test.ensemble64.csv')

In [ ]:
(acc, acc_with_prior), (f1, f1_with_prior) = eval_models.calcAccuracy_fullpage(learn_c, path, train_fullpage_df, valid_fullpage_df, test_ensemble_df, ensembled=True)
(acc, acc_with_prior), (f1, f1_with_prior)

### Error Analysis

In [ ]:
interp = ClassificationInterpretation.from_learner(learn_c)

In [ ]:
interp.plot_confusion_matrix(figsize=(12,12))